## First, let's test the ping method (GET /ping)

This method will be used by Sagemaker for health check our model. It must return a code **200**

In [ ]:
import json
from urllib import request

base_url='http://localhost:8080'

resp = request.urlopen("%s/ping" % base_url)
print("Response code: %d" % resp.getcode() )

## Then we can the predictions (POST /invocations)

This method will be used by Sagemaker for the predictions. Here we're simulating the header parameter related to the CustomAttributes

In [ ]:
%%time
import pandas as pd

def predict(test_df, content_type='text/csv', accept='text/csv'):
    headers = {
        'Content-type': content_type,
        'Accept': accept
    }
    
    if content_type == 'text/csv':
        payload = test_df.to_csv(index=False).encode('utf-8')
    else:
        payload = test_df.to_json().encode('utf-8')
        
    req = request.Request("%s/invocations" % base_url, data=payload, headers=headers)
    resp = request.urlopen(req)
    body = resp.read().decode('utf-8')
    print("Response code: %d, Prediction: %s\n" % (resp.getcode(), body))
    for i in resp.headers:
        print(i, resp.headers[i])

Get the first 10 records from test dataset

In [ ]:
# Get the first 10 records
test_df = pd.read_csv('test.csv').head(10)

Test all the different input and output formats

In [ ]:
predict(test_df, 'text/csv', 'text/csv')

In [ ]:
predict(test_df, 'text/csv', 'application/json')

In [ ]:
predict(test_df, 'application/json', 'application/json')

In [ ]:
predict(test_df, 'application/json', 'text/csv')